# Tools for auditing the Qdrant DB, finding and deleting documents


#### This is the **R** and **D** in the **CRUD** acronym. This notebook assumes Langchain `Document` object. See database_schema.md for more details. Qdrant documentation is located [here.](https://qdrant.github.io/qdrant/redoc/index.html#tag/points/operation/scroll_points). Adding to this becuase it's hard to find online searching


In [ ]:
%pip install --upgrade qdrant-client


In [2]:
import os, sys
import pandas as pd
from dotenv import load_dotenv

# Confirm correct interpreter is used
print(sys.executable)

# Add parent directory to sys.path to import modules from a subdirectory
sys.path.append(os.path.abspath('..'))


import library_utils as lib

/Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/.venv-311/bin/python


## 0. Imports and configs


In [3]:
import pandas as pd
from datetime import datetime


# Config Qdrant
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
# QDRANT_URL = st.secrets["QDRANT_URL"]
# QDRANT_API_KEY = st.secrets["QDRANT_API_KEY"]
QDRANT_PATH = "/Users/drew_wilkins/Drews_Files/Drew/Python/Localcode/Drews_Tools/qdrant_ASK_lib_tools/qdrant_db"

## 1. Connect to Qdrant


In [4]:
from qdrant_client import QdrantClient
from qdrant_client.http import exceptions as qdrant_exceptions


# Init client
# 22.5 sec for cloud
# Local client places a lock file in the qdrant directory
client = QdrantClient(
    url=QDRANT_URL,  # for cloud
    api_key=QDRANT_API_KEY,  # for cloud
    prefer_grpc=True,
    # path=QDRANT_PATH
)

lib.which_qdrant(client)
lib.list_collections(client)

qdrant location: cloud

collections:
ASK_vectorstore
ask_pdf_docs
ASK_vectorstore-backup21APR2025
ask_pdf_pages


In [5]:
# Read output to make sure you are accesing the right db in the right location

qdrant_collection_name = "ASK_vectorstore"

print(
    f"Number of points in {qdrant_collection_name} on {client._client}: {client.get_collection(qdrant_collection_name).points_count}")

Number of points in ASK_vectorstore on <qdrant_client.qdrant_remote.QdrantRemote object at 0x16beffe90>: 11615


## 3. Define a range index so you can sort on it
NOTE: expressions that use this index will not return records that are missing a page key

In [6]:
from qdrant_client.http import models
2.  # Creates a range index for `metadata.page`, so you can use this indexed field to order your results by page number.


client.create_payload_index(
    collection_name=qdrant_collection_name,
    field_name="metadata.page",
    # Assumes page numbers are stored as integers
    field_schema=models.PayloadSchemaType.INTEGER,
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

## 4. Find and delete records

1. Do an initial search on `metadata.source` based on the file name.
2. Review the initial search result to see if you get exactly what you want.
3. Iterate using `metadata.source` and `metadata.page_content` until you are sure.
4. Confirm it's giving you the doc, the whole doc, and nothing but the doc.
5. Delete away!


In [7]:
'''
This script uses a filter (scroll_filter) to count the number of points that match criteria and show the points. 

NOTE: The filter is case-sensitive and uses the `MatchText` condition. 
NOTE: This will not catch records lacking in a page field or a page field that lives in a different location.
Order_by requires qdrant-client 1.8 or later
'''

from qdrant_client.http import models

scroll_filter = models.Filter(
    must=[
        models.FieldCondition(
            # key="page_content",  # only in upsert>01NOV2024
            # key="metadata.pdf_id",  # only in upsert>01NOV2024
            # key="metadata.pdf_file_name",  # only in upsert>01NOV2024
            # key="metadata.page_content",  # pre-02NOV2024
            key="metadata.source",  # only usedful in pre-02NOV2024
            match=models.MatchText(
                text="Auxiliary_Division_Procedures_Guide_COMDTPUB"
            ),  # leave blank to get all
        ),
    ]
)

records = client.scroll(collection_name=qdrant_collection_name,
                        scroll_filter=scroll_filter,
                        with_payload=True,  # optional. Defaults to false
                        with_vectors=False,  # optional. Defaults to false
                        limit=10000,
                        order_by=models.OrderBy(
                            key="metadata.page",  # Field for ordering
                            direction="asc"       # "desc" for descending order
                        )
                        )

print(f"Number of points found: {len(records[0])}")
records[0]

Number of points found: 0


[]

#### Then uses the filter to **<span style="color:red">DELETE</span>** based on `metadata.source`


In [ ]:
from qdrant_client.http import models

client.delete(
    collection_name=qdrant_collection_name,
    points_selector=scroll_filter,
)

## OTHER EXAMPLES


### Example: Find points based on `source`


In [ ]:
from qdrant_client.http import models

records = client.scroll(
    collection_name=qdrant_collection_name,
    with_payload=True,  # change to True to see the payload
    with_vectors=False,  # change to True to see the vectors
    limit=100000,  # 👈 don't forget to set this

    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(
                # source is pdf_filepath in upsert<01NOV2024 afterwards its URL of pdf
                key="metadata.source",
                match=models.MatchText(
                    text="AUX-PL-017(A)"),
            ),
        ]
    ),
)

print(f"Number of points found: {len(records[0])}")
#records

### Example: Find points based on `scope`


In [ ]:
from qdrant_client.http import models
from qdrant_client.http.models import DatetimeRange

records = client.scroll(
    collection_name="ASK_vectorstore",
    with_payload=True,  # change to True to see the payload
    with_vectors=False,  # change to True to see the vectors
    limit=10, # 👈 don't forget to set this

    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="metadata.scope",
                match=models.MatchText(
                    text="1_national"),
            ),
        ]
    ),
)

print(f"Number of points found: {len(records[0])}")

Number of points found: 10


### Example: Find points based on `expiration_date`


In [ ]:
from qdrant_client.http import models
from qdrant_client.http.models import DatetimeRange
from datetime import datetime, timezone

records = client.scroll(
    collection_name="ASK_vectorstore",
    with_payload=True,
    with_vectors=False,
    limit=10000,  # 👈 don't forget to set this
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="metadata.expiration_date",
                range=DatetimeRange(gt=datetime.now(
                    timezone.utc).isoformat())  # datetime > today
            )
        ]
    )
)

print(f"Number of points found: {len(records[0])}")

Number of points found: 8834


### Example: Find points based on `public_release`  (a boolean)


In [7]:
from qdrant_client.http import models
from qdrant_client.http.models import DatetimeRange
from datetime import datetime, timezone

records = client.scroll(
    collection_name="ASK_vectorstore",
    with_payload=True,
    with_vectors=False,
    limit=1000,  # 👈 don't forget to set this
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="metadata.public_release",
                match=models.MatchValue(value=False)
            )
        ]
    )
)

print(f"Number of points found: {len(records[0])}")

Number of points found: 0


### Example: **<span style="color:red">Delete points based on the `source` as defined in THIS cell**</span>


In [ ]:
from qdrant_client.http import models

client.delete(
    collection_name=qdrant_collection_name,
    points_selector=models.Filter(
        must=[
            models.FieldCondition(
                key="metadata.source",
                match=models.MatchText(
                    text="_______"),  # BE CAREFUL CHECK THIS IS WHAT YOUR
            ),
        ]
    ),
)

print(f"Number of points deleted: {len(records[0])}")

### Example: **<span style="color:red">Delete points based on the `pdf_id` as defined in THIS cell**</span>


In [5]:
from qdrant_client.http import models

client.delete(
    collection_name=qdrant_collection_name,
    points_selector=models.Filter(
        must=[
            models.FieldCondition(
                key="metadata.pdf_id",
                match=models.MatchText(
                    text="20764d16-7066-5d44-9047-2cfa8a8fb888"),  # BE CAREFUL CHECK THIS IS WHAT YOUR
            ),
        ]
    ),
)

UpdateResult(operation_id=23214, status=<UpdateStatus.COMPLETED: 'completed'>)

### Example: **<span style="color:red">Delete points based on the `id` as defined in THIS cell**</span>


In [ ]:
from qdrant_client.http import models


client.delete(
    collection_name=qdrant_collection_name,
    points_selector=models.PointIdsList(
        points=["4cd5a55c-f37b-5c5d-b13d-b34d0a52305f"],
    ),
)

print(f"Number of points deleted: {len(records[0])}")

UpdateResult(operation_id=70666, status=<UpdateStatus.COMPLETED: 'completed'>)

### Example: Find points based on `id`


In [ ]:
from qdrant_client.http import models

records = client.scroll(
    collection_name=qdrant_collection_name,
    with_payload=True,  # change to True to see the payload
    with_vectors=False,  # change to True to see the vectors
    limit=100000,   # 👈 don't forget to set this
    scroll_filter=models.Filter(
        must=[
            models.HasIdCondition(
                has_id=["00cba8def26846769fd875fddf9f55bc"])
        ]
    )
)

print(f"Number of points found: {len(records[0])}")
records

Number of points found: 0


([], None)

### Example: Find points based on `pdf_id`

In [6]:
from qdrant_client.http import models

records = client.scroll(
    collection_name=qdrant_collection_name,
    with_payload=True,  # change to True to see the payload
    with_vectors=False,  # change to True to see the vectors
    limit=1000,  # 👈 don't forget to set this

    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="metadata.pdf_id",  # only in upsert>14NOV2024
                match=models.MatchText(
                    text="a9794b42-9ad4-5992-8dbe-29576740d623"),
            ),
        ]
    ),
)

print(f"Number of points found: {len(records[0])}")

Number of points found: 5


In [8]:
records[0][0].payload

{'metadata': {'upsert_date': '2025-04-22T01:12:54Z',
  'embedding': 'text-embedding-ada-002',
  'expiration_date': '2099-12-31T00:00:00Z',
  'organization': 'DCO',
  'scope': 'district',
  'pdf_file_name': 'D7-AUX-ID-Card-Request-Procedures.pdf',
  'issue_date': '2025-04-18T00:00:00Z',
  'public_release': True,
  'aux_specific': True,
  'publication_number': 'D7-SOP-AD-002',
  'pdf_id': 'a9794b42-9ad4-5992-8dbe-29576740d623',
  'source': './pdfs_backlog/D7-SOP-AD-002 Proctor Designation.pdf',
  'unit': '70.0',
  'title': 'Proctor Request and Designation Process Standard Operating Procedures',
  'page': 3},
 'page_content': 'District 7 Proctor Request and Designation Process \nD7-SOP-AD-002 4 18 April 2025 \n \n4) Responsible: Directors Administrative Aide (DAA) \n5) Procedure: \na. Receive the Help Desk ticket with the completed Proctor designation request. \nb. Validate member information in AUXDATA. \ni. AUXOP qualified \nii. Current in Core Training \niii. Current in Annual Uniform 

### Example: Find points that are missing the `page` field




In [ ]:
from qdrant_client.http import models

records = client.scroll(
    collection_name=qdrant_collection_name,
    with_payload=True,  # Set to True to see the payload
    with_vectors=False,  # Set to True to see the vectors
    limit=10000,  # 👈 don't forget to set this

    scroll_filter=models.Filter(
        must=[
            models.IsEmptyCondition(is_empty=models.PayloadField(key="metadata.page"),
                                    )
        ]
    )
)

print(f"Number of points found: {len(records[0])}")
records

Number of points found: 0


([], None)

### Example: Find points where `page` field does not exist or is inside the metadata dictionary!




In [ ]:
from qdrant_client.http import models

records = client.scroll(
    collection_name=qdrant_collection_name,
    with_payload=True,  # Set to True to see the payload
    with_vectors=False,  # Set to True to see the vectors
    limit=10000,  # 👈 don't forget to set this

    scroll_filter=models.Filter(
        must=[
            models.IsEmptyCondition(is_empty=models.PayloadField(key="page"),
                                    )
        ]
    )
)

print(f"Number of points found: {len(records[0])}")
records

### Example: Find points based on both `page` AND `source`




In [ ]:
from qdrant_client.http import models

records = client.scroll(
    collection_name=qdrant_collection_name,
    with_payload=True,  # change to True to see the payload
    with_vectors=False,  # change to True to see the vectors
    limit=1000,  # 👈 don't forget to set this

    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(
                # source is pdf_filepath in upsert<01NOV2024 afterwards its URL of pdf
                # key="metadata.source",
                # key="page_content",  # only in upsert>01NOV2024
                key="metadata.page_content",
                match=models.MatchText(
                    text=""),
            ),
            models.FieldCondition(
                key="metadata.page",
                match=models.MatchValue(
                    value=0),
            ),
        ]
    ),
)

print(f"Number of points found: {len(records[0])}")
records

### Example: Create a list of all unique file names 
(assuming they are not missing the page key)


In [ ]:
from qdrant_client.http import models

records = client.scroll(
    collection_name=qdrant_collection_name,
    with_payload=True,  # change to True to see the payload
    with_vectors=False,  # change to True to see the vectors
    limit=2,  # 👈 don't forget to set this

    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="metadata.page",
                match=models.MatchValue(
                    value=0),
            ),
        ]
    ),
)

print(f"Number of points found: {len(records[0])}")
records

In [ ]:
lib.close_qdrant(client)

closing client


### REFERENCE: Example record

```python

Record(
    id='01999d49fe0044478a1bffbb078938e8', 
    payload={
        'page_content': 'I Want a VSC\n•Revised “I Want a VSC” form is online with a new look.\n•Major changes are:\n•The originator no longer sees the selected VEs’ emails, only \nselected VEs see the emails.\n•Originator gets an email indicating they’ll be contacted.\n•Its’ important VEs “Reply All” they’ve made contact or aren’t \navailable.\n•If the originator does not receive a response -they wil be instructed \nto initiate another request.\n“I Want a VSC” can be found at:https://wow.uscgaux.info/i_want_a_vsc/index.php\n3\n٠The new “I Want A VSC” program is on-line.\n•Major changes are:\n•Originator no longer see VEs’ emails.\n•Originator receives an email indicating they’ll be contacted.\n•It’s important VEs “Reply All” they’ve made contact with originator or that \nthey’re not available.\n•If the originator hears nothing they’re instructed to initiate another request.\n•The new form is on line and can be found on the V-Directorate web page or with the \nQR code or web site.', 
        'metadata': {'source': './test_pdfs_copies_delete_after_use/VE_workshop_Dec_4_22_2023.pdf', 'page': 2, 'title': 'Auxiliary Vessel Examiner VE Workshop 2023', 'pdf_id': 'a0d00ff2-7ad5-5ea7-baa0-0d738380cd3e', 'publication_number': nan, 'organization': nan, 'scope': '1_national', 'issue_date': '2024-02-16T00:00:00Z', 'upsert_date': nan, 'expiration_date': '2034-02-16T00:00:00Z', 'aux_specific': True, 'public_release': True, 'embedding': 'text-embedding-ada-002', 'page_count': 31}
        }, 
    vector=None, 
    shard_key=None, 
    order_value=None
),
